In [1]:
import ROOT
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
bins = 32 
effective_bins = 24 

In [3]:
def pulse(peak_bin=8, adc_min=0, adc_max=800, bins=32, effective_bins=24,width_scale=0.5):
    
    
    pulse = np.random.uniform(0, 10, bins)

    
    f = ROOT.TF1("f", f"TMath::Landau(x / {width_scale})", 0, 50)
    x_vals = np.linspace(0,50, effective_bins)
    landau_tail = [f.Eval(x) for x in x_vals]

   
    landau_tail = landau_tail / np.max(landau_tail) * (adc_max - adc_min) + adc_min

    pulse[peak_bin:peak_bin+effective_bins] = landau_tail

    return pulse

In [4]:
muon_lifetime_ns = 2200
t_michel = np.random.exponential(scale=muon_lifetime_ns, size=int(125E4))
t_michel = t_michel[t_michel<15000]


noise1 = np.random.normal(loc=700, scale=100, size=int(500E2))
noise2 = np.random.normal(loc=1500, scale=200, size=int(500E2))
noise3 = np.random.normal(loc=1800, scale=150, size=int(500E2))
noise4 = np.random.normal(loc=4000, scale=250, size=int(700E2))
noise5 = np.random.normal(loc=11000, scale=400, size=int(500E2))

mask1  = (noise1>0)
mask2  = (noise2>0)
mask3  = (noise3>0)
mask4  = (noise4>0)
mask5  = (noise5>0)&(noise5 < 15000)

noise1 = noise1[mask1]
noise2 = noise2[mask2]
noise3 = noise3[mask3]
noise4 = noise4[mask4]
noise5 = noise5[mask5]

t_noise = np.concatenate([noise1, noise2, noise3,noise4, noise5])
np.random.shuffle(t_noise)

In [5]:
header = """# v 5
# #
# # This is a LAGO raw data file, version 5
# # It contains the following data:
# #   <N1> <N2>        : line with values of the 2 ADC for a triggered pulse
# #   # t <C> <V>      : end of a trigger
# #                      gives the channel trigger (<C>: 3 bit mask) and 125 MHz clock count (<V>) of the trigger time
# #   # c <C>          : internal trigger counter
# #   # x f <V>        : 125 MHz frequency
# #   # x t <V>        : temperature value
# #   # x p <V>        : pressure value
# #   # x r1 <V>       : pulse rate at channel 1
# #   # x r2 <V>       : pulse rate at channel 2
# #   # x h <HH:MM:SS> <DD/MM/YYYY> <S> : GPS time (every new second, last number is seconds since EPOCH)
# #   # x s <T> C <P> hPa <A> m : temperature <T>, pressure <P> and altitude (from pressure) <A>
# #   # x g <LAT> <LON> <ALT>   : GPS data - latitude, longitude, altitude
# #   # x v <HV1> <HV2>         : HV voltages for channels 1 and 2
# #   # x b <B1> <B2> <B3>      : baselines (NOT IMPLEMENTED IN LAGO)
# # In case of error, an unfinished line will be finished by # E @@@
# # Followed by a line with # E <N> and the error message in human readable format, where <N> is the error code:
# #   # E 1 : read timeout of 2 seconds
# #   # E 2 : too many buffer reading tries
# #   # E 3 : unknown word from FPGA
# #
# # Current registers setting
# #
# x c T1 600
# x c T2 8190
# x c HV1 1499.6 mV
# x c HV2 7.7 mV
# x c SC1 1
# x c SC2 1
# # This file was started on red-pitaya
# # WARNING, there is no GPS, using PC time
# #
# x h  
# x v 12.1 12.1
# p 4181140 -254.3 -240.6
# x p 767.95
# x t 23.5
# x f 124999999
# E @@@
# E 3 - unknown word from FPGA: 31 1f"""

In [7]:
total_seconds = 25
samples_per_second = 50
total_pulses = total_seconds * samples_per_second

output_path = Path("simulated_data.dat")
time_data = []


start_time = datetime.today()


cont_noise = 0
cont_signal = 0

df_data = []

    
with open(output_path, "w") as f:
    f.write(header + "\n")

    pulse_counter = 1
    current_time = start_time
    current_epoch = 1 
    accumulated_ticks = 0

    
    for i in range(total_pulses):
        if i % samples_per_second == 0:
            time_str = current_time.strftime("%H:%M:%S")
            date_str = current_time.strftime("%d/%m/%Y")
            f.write(f"# r2 0\n")
            f.write(f"# r1 200\n")
            f.write(f"# x g 0.000000 0.000000 0.00\n")
            f.write(f"# x h   {time_str} {date_str} {current_epoch} 0\n")
            f.write(f"# x v 12.1 12.1\n")
            f.write(f"# p 4181143 -254.3 -240.6\n")
            f.write(f"# x p 767.86\n")
            f.write(f"# x t 23.5\n")
            f.write(f"# x f 124999999\n")
            f.write(f"# E @@@\n")
            f.write(f"# E 3 - unknown word from FPGA: 31 1f\n")
            current_time += timedelta(seconds=1)
            current_epoch += 1


        is_noise = np.random.rand() < 0.8  # 80%  noise
        
        if is_noise:
                        
            is_sat = np.random.rand() > 1
            
            if is_sat:
                
                width_range=(20,30)
                
                width_scale_1 = np.random.uniform(*width_range)
                width_scale_2 = np.random.uniform(*width_range)
                
            else:
                width_range=(0.5,1)
                width_scale_1 = np.random.uniform(*width_range)
                width_scale_2 = np.random.uniform(*width_range)

            
            adc_max1 = np.random.randint(800, 1000)
            adc_max2 = np.random.randint(800, 1000)
                
            pulse1 = pulse(peak_bin=8, adc_min=0, adc_max=adc_max1, bins=32, effective_bins=18,width_scale=width_scale_1)
            pulse2 = pulse(peak_bin=8, adc_min=0, adc_max=adc_max2, bins=32, effective_bins=18,width_scale=width_scale_2)
    
            data = np.column_stack((pulse1.astype(int), pulse2.astype(int)))  
            
            
            
            ns_since_last = int(t_noise[cont_noise])
            cont_noise = 1 + cont_noise
            
            # data = {
            #    'label': int(not is_noise),
            #    'ns_since_last': ns_since_last}
            
            # df_data.append(data)
            
                      
            
        else:
            width_range=(1,2)
            
            adc_max1 = np.random.randint(800, 1000)
            adc_max2 = np.random.randint(800, 1000)
            
            width_scale_1 = np.random.uniform(*width_range)
            pulse1 = pulse(peak_bin=8, adc_min=0, adc_max=adc_max1, bins=32, effective_bins=20,width_scale=width_scale_1)
            
            width_scale_2 = np.random.uniform(*width_range)
            pulse2 = pulse(peak_bin=8, adc_min=0, adc_max=adc_max2, bins=32, effective_bins=20,width_scale=width_scale_2)
            
            data = np.column_stack((pulse1.astype(int), pulse2.astype(int)))  
            
            
            ns_since_last =  int(t_michel[cont_signal])
            cont_signal = cont_signal+1
            
            #data = {
            #    'label': int(not is_noise),
            #    'ns_since_last': ns_since_last}
            #df_data.append(data)
        
        
        time_data.append(ns_since_last)
        
        for row in data:
            f.write(f"{row[0]} {row[1]}\n")
        
        f.write(f"# t 1 {ns_since_last}\n")
        f.write(f"# c {pulse_counter}\n")
        

        pulse_counter += 1
        #print(pulse_counter)

        
# Descargar archivo
print(f"File Output: {output_path.resolve()}")
#df = pd.DataFrame(df_data)
#df.to_csv("puslses_record.csv", index=False)

File Output: /eos/home-i00/d/dmerizal/SWAN_projects/LAGO/simulated_data.dat
